# 数据集介绍

数据集类型是微博情感分类 

来源https://github.com/SophonPlus/ChineseNlpCorpus/blob/master/datasets/weibo_senti_100k/intro.ipynb

# 导入包

In [1]:
import sys,os
import pandas as pd
import random
import numpy as np
from nlp_basictasks.tasks import cls
from nlp_basictasks.evaluation import clsEvaluator
from nlp_basictasks.readers.cls import getExamplesFromData
import nlp_basictasks
print(nlp_basictasks.__version__)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
0.1.4


# 获取数据

In [2]:
data_path='/data/nfs14/nfs/aisearch/asr/xhsun/datasets/weibo_senti_100k.csv'
pd_all = pd.read_csv(data_path)

print('评论数目（总体）：%d' % pd_all.shape[0])
print('评论数目（正向）：%d' % pd_all[pd_all.label==1].shape[0])
print('评论数目（负向）：%d' % pd_all[pd_all.label==0].shape[0])

评论数目（总体）：119988
评论数目（正向）：59993
评论数目（负向）：59995


In [3]:
pd_all.sample(20)

,label,review
74624,0,苦逼了。撞车加失声，挂水还过敏。[泪]
98338,0,没检查到作业，百爪挠心啊[泪]
23380,1,"回复@一朵纯白的粒子云:西藏不是天堂,天堂也有乌云,我要做和要我做,仅仅是一步之遥[哈哈] ..."
55716,1,快出新品了哈[亲亲] //@Kaffee_Dora:看着好吃，今年就定这个? //@ebee...
100619,0,咋个一阵酸楚[泪][泪] //@赵楚: 他妈的！
22351,1,"Audi PR[威武],四环太有天缘了，Q3投放&奥迪俱乐部杯，什么也阻挡不了，大家快来吧 ..."
28719,1,#豆果趣事儿#[馋嘴]今天起豆果网有零食角啦！还是以进货价的一半的价格自助购买唷～亲们见到过...
58880,1,自然醒！早[可爱]
75755,0,//@frank7383: //@W天之蓝W:这个总统应该上断头台[怒] //@frank7...
50973,1,记得小时候被人欺负了，我就在家里磨菜刀，并豪言：谁尼玛再欺负我，我非剁了他！ //@长话短说...


## 打乱数据集，同时划分训练、验证

In [4]:
print(len(pd_all))
random_idx=np.random.permutation(len(pd_all))

119988


In [5]:
#时间关系仅取前20000个作为训练
sentences=pd_all['review'].values[random_idx].tolist()[:20000]
labels=pd_all['label'].values[random_idx].tolist()[:20000]
print(len(sentences),len(labels))
random_idx=np.random.permutation(len(sentences))

20000 20000


In [6]:
label2id={'0':0,'1':1}
dev_ratio=0.2#训练集的20%作为验证
dev_nums=int(len(sentences)*dev_ratio)
train_nums=len(sentences)-dev_nums
print(dev_nums)
train_sentences=sentences[:train_nums]
train_labels=labels[:train_nums]
dev_sentences=sentences[-dev_nums:]
dev_labels=labels[-dev_nums:]
train_examples,max_seq_len=getExamplesFromData(sentences=train_sentences,labels=train_labels,label2id=label2id,mode='train',return_max_len=True)
dev_examples=getExamplesFromData(sentences=dev_sentences,labels=dev_labels,label2id=label2id,mode='dev')

2021-08-25 18:11:32 - INFO - getExamplesFromData - 128 : *****************************Logging some train examples*****************************
2021-08-25 18:11:32 - INFO - getExamplesFromData - 129 : Total train nums is : 16000
2021-08-25 18:11:32 - INFO - getExamplesFromData - 132 : 
//@神笔记:把人快折腾残废了，1000多块就了事？[怒]	0
2021-08-25 18:11:32 - INFO - getExamplesFromData - 132 : 
#五月专属优惠#即日起，关注咖啡馆微信，点很多人的任一种类?饭就送价值23元蜂蜜柚子茶一杯哦，超值套餐巨划算！[太开心] 仅限现金消费~	1
2021-08-25 18:11:32 - INFO - getExamplesFromData - 132 : 
//@诗人潘婷: 这畜生！绝对嚣张不止一日了！为何会长期呆在警察的队伍里？[怒]//@陈明远微博:[怒]//@徐昕: [怒]//@声音法治周刊:[怒]现在的土匪……//@周永坤微博:警察开枪就为了耍威风？	0
2021-08-25 18:11:32 - INFO - getExamplesFromData - 132 : 
[泪][泪] //@丛中笑327:[偷笑] //@徐巍:一样一样的[偷笑] //@王潮歌:我马上给孩子他爸看了！	0
2021-08-25 18:11:32 - INFO - getExamplesFromData - 132 : 
我手机满屏了等我清理一下 //@day11: 姑娘 你太神了！[good]我昨天爱拍得里下了五篇民宿介绍占了2个多G……我们去台湾用3G网嗖嗖的下！[偷笑] 转给爱疯小王子他媳妇 @michelle75005 还有爱疯小王子本人@land浪摄流   [哈哈][哈哈]各取所需吧！	1
2021-08-25 18:11:32 - INFO - getExamplesFromData - 128 : ****************

4000


In [9]:
train_examples[:4]

# 定义路径加载模型

In [8]:
#max_seq_len就是训练集中最长的句子长度
model_path='/data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/'
print(max_seq_len)
max_seq_len=min(512,max_seq_len)
cls_model=cls(model_path=model_path,label2id=label2id,max_seq_length=max_seq_len,device='cpu')

2021-08-24 19:43:49 - INFO - __init__ - 48 : The label2id is
 {"0": 0, "1": 1}
2021-08-24 19:43:49 - INFO - __init__ - 58 : Loading model from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/, which is from huggingface model
2021-08-24 19:43:49 - INFO - get_config_dict - 177 : loading configuration file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json
2021-08-24 19:43:49 - INFO - from_pretrained - 404 : loading bert model file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/
2021-08-24 19:43:49 - INFO - from_pretrained - 423 : BertConfig has been loaded from /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/config.json


256


2021-08-24 19:43:51 - INFO - from_pretrained - 125 : loading vocabulary file /data/nfs14/nfs/aisearch/asr/xhsun/CommonModel/chinese-roberta-wwm/vocab.txt


In [10]:
from torch.utils.data import DataLoader
batch_size=32
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=batch_size)

# 定义evaluator

In [11]:
evaluator=clsEvaluator(sentences=dev_sentences,label_ids=dev_labels,write_csv=False,label2id=label2id)

2021-08-24 19:44:28 - INFO - __init__ - 28 : label2id like : {'0': 0, '1': 1}
2021-08-24 19:44:28 - INFO - __init__ - 39 : The number of 0 in dataset is 2020
2021-08-24 19:44:28 - INFO - __init__ - 39 : The number of 1 in dataset is 1980
2021-08-24 19:44:28 - INFO - __init__ - 45 : Evalautor sentence like : 

2021-08-24 19:44:28 - INFO - __init__ - 47 : [衰] //@中原网:【图片中的“海市蜃楼”系周边建筑】有结果啦，虚惊一场，有多位网友向小编反映，下午的时候这个点经过该路口，并未发现有“海市蜃楼”现象出现，图片上的“海市蜃楼”应该是对面的建筑（经现场核实，系附近一楼盘）反射在玻璃板上，正好被拍下来了，拍摄者当时可能坐在公交车里拍的。	0

2021-08-24 19:44:28 - INFO - __init__ - 47 : @一辰 ，尼玛太有才了[cai正呀] [哈哈] @隋杨	1

2021-08-24 19:44:28 - INFO - __init__ - 47 : 小伙伴儿们现在拿起手机，打开微博客户端-广场，双节有惊喜呦！[吃元宵][moc亲吻]//@兰亦兰: [蜡烛][泪]//@我不是蛐蛐: [哈哈] //@郭宏:转发微博	1

2021-08-24 19:44:28 - INFO - __init__ - 47 : 我们饮中八仙也郑重承诺 剩下的画展 我们必须都去 哈哈//@大匠之门李笑天: 回复@八年小散:我郑重诚诺。要在中国美术办它10次画展[嘻嘻][嘻嘻] //@八年小散:还有听见此话的吗？ //@大匠之门李笑天:[嘻嘻][嘻嘻][威武] //@郭月晨:这个我听见了 //@苗雪珊: [可爱]真滴嘛	1

2021-08-24 19:44:28 - INFO - __init__ - 47 : [蜡烛][蜡烛][蜡烛]求报复社会的人们放过无辜老百姓吧[伤心][伤心][伤心]	0



# 训练模型
- is_pairs用来指明是单句子分类任务还是双句子分类任务
- output_path是保存模型的路径

In [12]:
cls_model.fit(is_pairs=False,train_dataloader=train_dataloader,evaluator=evaluator,output_path="/data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/")

2021-08-24 19:45:35 - INFO - fit - 102 : 当前是单句子分类任务
2021-08-24 19:45:35 - INFO - fit - 113 : 一个epoch 下，每隔100个step会输出一次loss，每隔250个step会评估一次模型


2021-08-24 19:45:43 - INFO - __call__ - 63 : ClsEvaluator: Evaluating the model on  dataset in epoch 0 after 1 steps:
2021-08-24 19:45:43 - INFO - predict - 221 : 当前是单句子分类任务预测


2021-08-24 19:49:18 - INFO - __call__ - 70 : Accuracy: 0.436
2021-08-24 19:49:18 - INFO - __call__ - 82 : 0	precision : 0.450293, recall : 0.529205,  f1 score : 0.481653
2021-08-24 19:49:18 - INFO - __call__ - 82 : 1	precision : 0.415127, recall : 0.340907,  f1 score : 0.369487
2021-08-24 19:49:18 - INFO - __call__ - 93 : AUC: 0.404


2021-08-24 19:49:20 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 19:49:20 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 19:49:20 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 19:49:20 - INFO - fit - 188 : In epoch 0, training_step 0, the eval score is 0.403772502250225, previous eval score is -9999999, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/
2021-08-24 20:00:12 - INFO - fit - 166 : Epoch : 0, train_step : 100/500, loss_value : 0.341104484423995 
2021-08-24 20:10:16 - INFO - fit - 166 : Epoch : 0, train_step : 200/500, loss_value : 0.06391732832184062 
2021-08-24 20:15:35 - INFO - __call__ - 63 : ClsEvaluator: Evaluating the model on  dataset in epoch 0 after 251 steps:
2021-08-24 20:15:35 - INFO - predict - 221 : 当前是单句子分

2021-08-24 20:18:48 - INFO - __call__ - 70 : Accuracy: 0.981
2021-08-24 20:18:48 - INFO - __call__ - 82 : 0	precision : 0.965529, recall : 0.998510,  f1 score : 0.976769
2021-08-24 20:18:48 - INFO - __call__ - 82 : 1	precision : 0.998425, recall : 0.963631,  f1 score : 0.975747
2021-08-24 20:18:48 - INFO - __call__ - 93 : AUC: 0.998


2021-08-24 20:18:55 - INFO - save_pretrained - 509 : Model weights saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/pytorch_model.bin
2021-08-24 20:18:55 - INFO - save_pretrained - 150 : Configuration saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/config.json
2021-08-24 20:18:55 - INFO - save_vocab - 51 : Vocab saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/BERT/vocab.txt
2021-08-24 20:18:55 - INFO - fit - 188 : In epoch 0, training_step 250, the eval score is 0.9983108310831084, previous eval score is 0.403772502250225, model has been saved in /data/nfs14/nfs/aisearch/asr/xhsun/tmp_model/
2021-08-24 20:23:54 - INFO - fit - 166 : Epoch : 0, train_step : 300/500, loss_value : 0.04959781204350293 


KeyboardInterrupt: 

**时间关系暂停训练**

# 测试模型

In [16]:
predict_probs=cls_model.predict(is_pairs=False,dataloader=['这孩子真可爱','这人看起来像傻子似的'])
print(predict_probs)
id2label={id_:label for label,id_ in label2id.items()}
predict_tags=[id2label[id_] for id_ in np.argmax(predict_probs,axis=1)]
print(predict_tags)

2021-08-24 20:29:10 - INFO - predict - 221 : 当前是单句子分类任务预测



[[0.00504928 0.9949508 ]
 [0.6484326  0.35156742]]
['1', '0']
